# Contact Center Insights using Generative AI

Most financial institutions have contact centers where customers can call in and ask for help. When a contact center agent connects with a customer they are asked to follow a set of guidelines and templates they need to follow. For financial services in certain countries there are regulations that agents need to follow.

After each call agents need to perform certain actions: 

* produce a summary of the call and enter it into an internal CRM such as Salesforce or a case management or ticketing system
* write an email correspondence with a summary of the call and list out the actions discussed


Additionally, the call needs to reviewed by team leads and auditors to ensure that company standards and regulations are met. These actions include:
* perform QA check
* capture customer sentiment after the call


## Solution Architecture

![Workflow for analyzing post-call recordings](./9.1.jpg)

Figure 9.1: Workflow showing steps involved in analyzing post-call recordings

In this example, we will start with an example call transcript. For simplicity, we will skip the call transcription step assuming it has already happened. We will focus on how to use GenAI to perform various post-call analysis steps we discussed above.


Let's take a look at the sample call transcript

In [ ]:
# Installing required libraries
!pip3 install --upgrade boto3

In [1]:
with open("./bank-call-centre-transcript.txt") as f:
    transcript = f.read()

print(transcript)

Agent: Hello, this is Lisa from National Bank. How are you today? 
Customer: I'm good. Thank you. I'm calling in relation to a problem I'm having with my account.
Agent: I'm sorry to hear that. Before we begin, let me inform you that this call will be recorded for quality training and regulatory purposes. Is that ok? 
Customer: Yeah, that's ok. 
Agent: Great. Thank you. What's the problem you are facing?
Customer: I am not able to login to my account and also not able to use my debit card. I have tried resetting my password but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to the website and try resetting the password from there and see if you get password reset email and try following the steps from there?
Customer: Ok, i tried it and received the password reset mail. I have reset my password but the login still fails.
Agent: Okay, from your login history i can see you tried few times last week and your account got locked out. You al

In [4]:
# Utility function to call Llama 3 model 
import boto3


def generate_conversation(bedrock_client,
                     model_id,
                     system_text,
                     input_text):
    """
    Sends a message to a model.
    Args:
        bedrock_client: The Boto3 Bedrock runtime client.
        model_id (str): The model ID to use.
        system_text (JSON) : The system prompt.
        input text : The input message.

    Returns:
        response (JSON): The conversation that the model generated.

    """

    print("Generating message with model", model_id)

    # Message to send.
    message = {
        "role": "user",
        "content": [{"text": input_text}]
    }
    messages = [message]
    system_prompts = [{"text" : system_text}]

    # Inference parameters to use.
    temperature = 0.1

    #Base inference parameters to use.
    inference_config = {"temperature": temperature}

    # Send the message.
    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompts,
        inferenceConfig=inference_config,
    )

    return response


def ask_llm(system_text, input_text):
    """
    Generates text using a large language model (LLM).

    Args:
        prompt (str): The input prompt to generate text from.

    Returns:
        str: The generated text.
    """

    model_id = "meta.llama3-70b-instruct-v1:0"
    aws_region = "us-east-1"
    bedrock = boto3.client(service_name='bedrock-runtime', region_name=aws_region)

    response = generate_conversation(bedrock, model_id, system_text, input_text)
    answers = response['output']['message']['content']
    return answers


## Use-case 1: Call Summarization
One common requirement for contact center agents is to create a detailed summarize the customer call which is recorded within various internal systems. We can use LLMs to summarize the call transcript using prompt engineering techniques. We have an example prompt below:

In [5]:
system_prompt_template = """
  Inside the <transcript></transcript> XML tags is a transcript from a phone call between a Banking Customer Advisor (also known as the Agent) and a Customer (also know as the Customer) regarding an online website related issue.

  <transcript>$transcript</transcript>

  Use only this transcript for context.
"""

summary_prompt = """
  Your task is to write a detailed summary of the call in dot point form, from the perspective of the Agent. Include a line break between each dot point.

  Respond immediately without preamble, and output the response in the following example format:
  
  Transcript Summary:
  - (summary dot points)

  Assistant: OK, here is the summary:
"""

from string import Template
system_prompt = Template(system_prompt_template).safe_substitute(transcript=transcript)

output_lines = ask_llm(system_prompt, summary_prompt)
for line in output_lines:
    print(line['text'])


Generating message with model meta.llama3-70b-instruct-v1:0


Transcript Summary:
 - I answered the call and introduced myself as Lisa from National Bank, ensuring the customer was aware that the call would be recorded for quality training and regulatory purposes.

 - The customer explained they were experiencing issues with their account, specifically being unable to log in and use their debit card, despite attempting to reset their password.

 - I guided the customer through troubleshooting steps, asking them to reset their password from the website and follow the password reset email instructions.

 - The customer reported that they had received the password reset email but were still unable to log in.

 - I reviewed the customer's login history and found that their account had been locked out due to multiple failed login attempts, and their debit card had been blocked due to incorrect PIN attempts.

 - The customer confirmed the failed login attempts but denied using their debit ca

## Use-case 2: Creating case notes
After a customer call, contact center agents often need to record the key points and notes in internal systems such as CRMs. Let's see how we can use prompt engineering to generate case notes. 

In [6]:

case_note_prompt = """
  Write a concise summary of the call in dot point form. Be sure to:
  1. Summarise the nature of the injury.
  2. Any administrative steps being undertaken by the Agent.
  3. Any follow up actions or next steps that were mentioned.
  4. Write in 1st person from the perspective of the Agent.
  5. Include a line break between each dot point.
  6. Do not include preamble in your response. 

  Respond immediately without preamble, and output the response in the following format:

  Case: (one sentence summary of the nature of the injury)

  Notes:
  - (include dot point summary of the nature of the injury, treatment, and current work circumstances)

  Actions:
  - (include dot point summary of any administrative steps, follow up actions, or next steps being undertaken by the Agent)
"""

output_lines = ask_llm(system_prompt, case_note_prompt)
for line in output_lines:
    print(line['text'])


Generating message with model meta.llama3-70b-instruct-v1:0


Case: Customer is experiencing issues with logging into their online account and using their debit card.

Notes:
  - Customer is unable to login to their online account and use their debit card despite resetting their password.

  - Customer's account was locked out due to multiple incorrect login attempts and debit card was blocked due to incorrect PIN attempts.

  - Unauthorized debit card transactions were detected in Eastern Europe.

Actions:
  - Permanently blocking the customer's debit card and issuing a new one to be mailed to the customer.

  - Resetting the customer's account credentials from the backend due to potential account compromise.

  - Customer will receive an email with temporary login credentials within the next hour, which will prompt them to create a new password.

  - Customer will receive the new debit card and PIN by mail within 5 business days.


## Use-case 3: Email correspondence
After the call is finished, agents typically are required to summarize the discussion and confirm the follow-up actions in an email back to the customer. Here we see how we could generate an email correspondence using prompt engineering. 

In [7]:

correspondence_prompt = """
  Draft a email from the Agent to the Customer summarising the conversation. Be sure to:
  1. Thank the customer for their time
  2. Highlight the issue discussed and recommended actions 
  3. Any administrative steps being undertaken by the Agent
  4. Any follow up actions that were agreed upon
  5. Use sentences not dot points.
  6. Sign the letter off from National Bank

  The tone of the letter should be personable, but professional. 

  Respond immediately without preamble.

  Assistant: OK, here is the email:
"""

output_lines = ask_llm(system_prompt, correspondence_prompt)
for line in output_lines:
    print(line['text'])


Generating message with model meta.llama3-70b-instruct-v1:0


Subject: Summary of Our Conversation Regarding Your Account Issue

Dear [Customer's Name],

I wanted to take a moment to thank you for reaching out to us about the issues you've been experiencing with your online account and debit card. I apologize again for the inconvenience this has caused and appreciate the time you took to speak with me today.

As we discussed, it appears that your account was locked out due to multiple failed login attempts, and your debit card was blocked after several incorrect PIN entries. Additionally, our system detected unauthorized usage on your debit card in Eastern Europe, which we take very seriously.

To resolve these issues, I have taken steps to permanently block your current debit card and will be issuing a new one, which you should receive by mail within the next 5 business days. Your new debit card PIN will be mailed separately. I have also reset your account credentials from the backend

## Use-case 4: Post-call Quality Checks
Contact center agents have to follow various regulatory requirements and enterprise quality standards when engaging with customers. These requirements need to be validate post-call to ensure that standards are upheld. This is a highly manual task and because of the high volume in most contact centers this can't be performed on all calls, and only a sample of the calls are analysed. 

With the new GenAI techniques, we can automatically validate all calls based on our requirements and standards. Let's see how we can achieve this using prompt engineering. 

In [20]:

qa_prompt = """
  Your task is to analyse the transcript and answer each of questions inside the <qa></qa> XML tags.  

  <qa>
  Did the Agent identify themself?
  Did the Agent explicitly state they were conducting a 'privacy check'?
  Did the Agent mention that the call is being recorded?
  Confirmed the nature of the customer problem?
  Confirmed the customer preferred method of contact?
  Communicated follow-up actions?
  </qa>

  Answer with [Y] if the question can be answered (yes), or [N] if it cannot (no). 

  Based on your answer, calculate a score percentage. The formula for calculating the score percentage is:
  <formula>
  (number of questions answered with [Y]) / (total number of questions)
  </formula>

  Do not output in XML.
"""

output_lines = ask_llm(system_prompt, qa_prompt)
for line in output_lines:
    print(line['text'])


Generating message with model meta.llama3-70b-instruct-v1:0


Here are the answers to the questions:

1. Did the Agent identify themself? - Y
2. Did the Agent explicitly state they were conducting a 'privacy check'? - N
3. Did the Agent mention that the call is being recorded? - Y
4. Confirmed the nature of the customer problem? - Y
5. Confirmed the customer preferred method of contact? - N
6. Communicated follow-up actions? - Y

Score percentage: (4/6) * 100% = 66.67%


## Use-case 5: Question answering based on particular call 
There is sometimes a requirement of further analysis and investigation on a call, for e.g. if there is a customer escalation. Call transcripts can be very long and hard to read in raw format. 

To help us with QnA on a call, we can use LLMs to analyse and answer questions on a call transcript as below.

In [21]:
insights_prompt_template = """
Answer the following question: $question
"""

question = "what was the customer problem?"

insights_prompt = Template(insights_prompt_template).safe_substitute(question=question)

output_lines = ask_llm(system_prompt, insights_prompt)
for line in output_lines:
    print(line['text'])



Generating message with model meta.llama3-70b-instruct-v1:0


The customer problem was that they were unable to log in to their online account and were also unable to use their debit card. They had tried resetting their password, but the issue persisted.
